In [1]:
import pandas as pd 
import numpy as np
import re
import warnings
warnings.filterwarnings("ignore")

In [2]:
fifa_data=pd.read_csv('fifa21 raw data v2.csv')
df=fifa_data.copy()

In [3]:
df.sample(10)

,ID,Name,LongName,photoUrl,playerUrl,Nationality,Age,↓OVA,POT,Club,...,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
5303,221912,Kang Hyeon Mu,Hyeon Mu Kang,https://cdn.sofifa.com/players/221/912/21_60.png,http://sofifa.com/player/221912/hyeon-mu-kang/...,Korea Republic,25,70,74,\n\n\n\nPohang Steelers,...,Medium,Medium,1 ★,71,64,63,74,45,69,3
6916,213404,P. Dawidowicz,Paweł Dawidowicz,https://cdn.sofifa.com/players/213/404/21_60.png,http://sofifa.com/player/213404/pawel-dawidowi...,Poland,25,68,74,\n\n\n\nHellas Verona,...,High,High,1 ★,59,46,61,61,68,74,8
5521,190747,Ortuño,Alfredo Ortuño Martínez,https://cdn.sofifa.com/players/190/747/21_60.png,http://sofifa.com/player/190747/alfredo-ortuno...,Spain,29,69,69,\n\n\n\nAlbacete BP,...,Medium,Low,1 ★,64,71,55,62,31,69,3
14049,256884,J. Rahou,Jorès Rahou,https://cdn.sofifa.com/players/256/884/21_60.png,http://sofifa.com/player/256884/jores-rahou/21...,France,17,61,74,\n\n\n\nOlympique de Marseille,...,High,Low,1 ★,61,58,63,60,47,53,18
6121,214208,L. Pico,Leonardo Pico,https://cdn.sofifa.com/players/214/208/21_60.png,http://sofifa.com/player/214208/leonardo-pico/...,Colombia,28,69,69,\n\n\n\nJunior FC,...,Medium,High,1 ★,58,45,64,65,63,75,2
17819,254260,K. Egell-Johnsen,Kevin Egell-Johnsen,https://cdn.sofifa.com/players/254/260/21_60.png,http://sofifa.com/player/254260/kevin-egell-jo...,Norway,20,55,70,\n\n\n\nOdds BK,...,Medium,Medium,1 ★,66,26,36,52,51,57,NaN
2163,202054,Naldo,Edinaldo Gomes Pereira,https://cdn.sofifa.com/players/202/054/21_60.png,http://sofifa.com/player/202054/edinaldo-gomes...,Brazil,31,74,74,\n\n\n\nAntalyaspor,...,Medium,Medium,1 ★,37,38,48,51,77,74,4
5147,221621,S. Birnbaum,Steve Birnbaum,https://cdn.sofifa.com/players/221/621/21_60.png,http://sofifa.com/player/221621/steve-birnbaum...,United States,29,70,71,\n\n\n\nDC United,...,Low,High,1 ★,38,31,46,51,69,80,7
1900,244206,N. Pérez,Nehuén Pérez,https://cdn.sofifa.com/players/244/206/21_60.png,http://sofifa.com/player/244206/nehuen-perez/2...,Argentina,20,75,85,\n\n\n\nGranada CF,...,Medium,High,1 ★,63,25,46,44,77,74,143
9796,202799,J. Graham,Jordan Graham,https://cdn.sofifa.com/players/202/799/21_60.png,http://sofifa.com/player/202799/jordan-graham/...,England,25,65,68,\n\n\n\nGillingham,...,Medium,Low,1 ★,77,58,61,68,27,47,7


# First Look

In [4]:
def show_missing(df):
    """Return a Pandas dataframe describing the contents of a source dataframe including missing values."""
    
    variables = []
    dtypes = []
    count = []
    unique = []
    missing = []
    pc_missing = []
    
    for item in df.columns:
        variables.append(item)
        dtypes.append(df[item].dtype)
        count.append(len(df[item]))
        unique.append(len(df[item].unique()))
        missing.append(df[item].isna().sum())
        pc_missing.append(round((df[item].isna().sum() / len(df[item])) * 100, 2))

    output = pd.DataFrame({
        'variable': variables, 
        'dtype': dtypes,
        'count': count,
        'unique': unique,
        'missing': missing, 
        'pc_missing': pc_missing
    })    
        
    return output.sort_values("missing",ascending=False).reset_index(drop=True)

In [5]:
summary_df = show_missing(df)
summary_df

,variable,dtype,count,unique,missing,pc_missing
0,Loan Date End,object,18979,25,17966,94.66
1,Hits,object,18979,451,2595,13.67
2,GK Diving,int64,18979,69,0,0.00
3,Sliding Tackle,int64,18979,85,0,0.00
4,Standing Tackle,int64,18979,87,0,0.00
...,...,...,...,...,...,...
72,Heading Accuracy,int64,18979,89,0,0.00
73,Finishing,int64,18979,93,0,0.00
74,Crossing,int64,18979,89,0,0.00
75,Attacking,int64,18979,367,0,0.00


In [6]:
df[df.duplicated()]

,ID,Name,LongName,photoUrl,playerUrl,Nationality,Age,↓OVA,POT,Club,...,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits


<div style=" background-color:#3b3745;text-align:left; padding: 13px 13px; border-radius: 8px; color: white">
<ul>
<li> No duplicated date.
<li> There is some missing data
</ul>
</div>

# Columns Name Change

In [7]:
df.keys()

Index(['ID', 'Name', 'LongName', 'photoUrl', 'playerUrl', 'Nationality', 'Age',
       '↓OVA', 'POT', 'Club', 'Contract', 'Positions', 'Height', 'Weight',
       'Preferred Foot', 'BOV', 'Best Position', 'Joined', 'Loan Date End',
       'Value', 'Wage', 'Release Clause', 'Attacking', 'Crossing', 'Finishing',
       'Heading Accuracy', 'Short Passing', 'Volleys', 'Skill', 'Dribbling',
       'Curve', 'FK Accuracy', 'Long Passing', 'Ball Control', 'Movement',
       'Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance',
       'Power', 'Shot Power', 'Jumping', 'Stamina', 'Strength', 'Long Shots',
       'Mentality', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure', 'Defending', 'Marking', 'Standing Tackle',
       'Sliding Tackle', 'Goalkeeping', 'GK Diving', 'GK Handling',
       'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Total Stats',
       'Base Stats', 'W/F', 'SM', 'A/W', 'D/W', 'IR', 'PAC', 'SHO', 'PAS',
       'DRI', 'DEF', 

In [8]:
def clean_column_name(df):
    column_name=[]
    for col  in df.columns:
        clean_col = re.findall("[a-zA-Z]", col)
        clean_col = ''.join(clean_col)
        clean_col = clean_col.lower()
        column_name.append(clean_col)
           
        
    return  column_name  

In [9]:
df.columns=clean_column_name(df)

In [10]:
df.keys()

Index(['id', 'name', 'longname', 'photourl', 'playerurl', 'nationality', 'age',
       'ova', 'pot', 'club', 'contract', 'positions', 'height', 'weight',
       'preferredfoot', 'bov', 'bestposition', 'joined', 'loandateend',
       'value', 'wage', 'releaseclause', 'attacking', 'crossing', 'finishing',
       'headingaccuracy', 'shortpassing', 'volleys', 'skill', 'dribbling',
       'curve', 'fkaccuracy', 'longpassing', 'ballcontrol', 'movement',
       'acceleration', 'sprintspeed', 'agility', 'reactions', 'balance',
       'power', 'shotpower', 'jumping', 'stamina', 'strength', 'longshots',
       'mentality', 'aggression', 'interceptions', 'positioning', 'vision',
       'penalties', 'composure', 'defending', 'marking', 'standingtackle',
       'slidingtackle', 'goalkeeping', 'gkdiving', 'gkhandling', 'gkkicking',
       'gkpositioning', 'gkreflexes', 'totalstats', 'basestats', 'wf', 'sm',
       'aw', 'dw', 'ir', 'pac', 'sho', 'pas', 'dri', 'def', 'phy', 'hits'],
      dtype='obje

<div style=" background-color:#3b3745;text-align:left; padding: 13px 13px; border-radius: 8px; color: white">
<ul>
<li> I generally use column names in lowercase, without spaces and free from unusual expressions.
<li> However, it is optional for you to do so
</ul>
</div>

# Clean Columns

<div style=" background-color:#3b3745;text-align:left; padding: 13px 13px; border-radius: 8px; color: white">
<ul>
<li>'attacking’, 'crossing', 'finishing’, 'headingaccuracy', 'shortpassing’, 'volleys’, 'skill’, 'dribbling’, 'curve’, 'fk accuracy’, 'longpassing', 'ballcontrol’, 'movement’, 'acceleration’, 'springspeed’, 'agility’, 'reactions’, 'balance’, 'power’, 'shotpower’, 'jumping’, 'stamina’, 'strength’, 'longshot', 'mentality’, 'aggression’, 'interception’, 'positioning’, 'vision’, 'penalties’, 'composure’, 'defending’, 'marking’, 'standingtackle’, 'slidingtackle’, 'goalkeeping’, 'gkhandling’, 'gkdiving’, 'gkkicking’, 'gkpositioning’, 'gkreflexes’, 'totalstats’, 'basestats’, 'pace', 'shooting’, 'passing’, 'dri’, 'def’, 'physical’, 'bestposition’, 'Preferred_foot’, 'attackingworkrate’,'positions', and 'defensiveworkrate'
<li> were checked and no issues found within the data.
</ul>
</div>



# Dirty Columns

## Name Longname Playerurl

In [11]:
df.loc[:,['name','longname','playerurl']].sample(10)

,name,longname,playerurl
9747,J. Chibuike,John Chibuike,http://sofifa.com/player/192285/john-chibuike/...
5056,A. McGeady,Aiden McGeady,http://sofifa.com/player/155885/aiden-mcgeady/...
9549,J. Skjelvik,Jørgen Skjelvik,http://sofifa.com/player/188917/jorgen-skjelvi...
17032,Luo Yi,Yi Luo,http://sofifa.com/player/247196/yi-luo/210006/
17956,S. Gefvert,Simon Gefvert,http://sofifa.com/player/256369/simon-gefvert/...
10862,K. Ofori,Kelvin Ofori,http://sofifa.com/player/252438/kelvin-ofori/2...
2166,João Pereira,João Pedro da Silva Pereira,http://sofifa.com/player/206407/joao-pedro-da-...
12826,Álvaro García,Álvaro García Segovia,http://sofifa.com/player/243156/alvaro-garcia-...
15772,J. Lokilo,Jason Lokilo,http://sofifa.com/player/231267/jason-lokilo/2...
2999,L. Gil,Leonardo Gil,http://sofifa.com/player/216020/leonardo-gil/2...



<div style=" background-color:#3b3745;text-align:left; padding: 13px 13px; border-radius: 8px; color: white">
<ul>
<li> Here are special characters in the name and long name columns that may cause issues. 
<li> Therefore, I will retrieve the names from the playerurl column, where the names are formatted better
</ul>
</div>

In [12]:
def clean_name(dataframe,col):
    clean_name = []
    for name in dataframe[col]:
        name=name.split("/")[5].replace("-",' ').title()
        clean_name.append(name) 
    return pd.DataFrame(clean_name,columns=['fullname'])      

In [13]:
clean_name(df,'playerurl')

,fullname
0,Lionel Messi
1,C Ronaldo Dos Santos Aveiro
2,Jan Oblak
3,Kevin De Bruyne
4,Neymar Da Silva Santos Jr
...,...
18974,Ao Xia
18975,Ben Hough
18976,Ronan Mckinley
18977,Zhenao Wang


In [14]:
df['fullname']=clean_name(df,'playerurl')

In [15]:
df.loc[:,['name','longname','playerurl','fullname']].sample(10)

,name,longname,playerurl,fullname
2779,J. Bela,Jérémie Bela,http://sofifa.com/player/212035/jeremie-bela/2...,Jeremie Bela
11945,K. Dempsey,Kyle Dempsey,http://sofifa.com/player/221705/kyle-dempsey/2...,Kyle Dempsey
8006,O. Brynhildsen,Ola Brynhildsen,http://sofifa.com/player/238236/ola-brynhildse...,Ola Brynhildsen
17022,S. Todd,Sam Todd,http://sofifa.com/player/235704/sam-todd/210006/,Sam Todd
15643,A. Karagülmez,Akif Karagülmez,http://sofifa.com/player/257826/akif-karagulme...,Akif Karagulmez
17309,T. Leysen,Tobe Leysen,http://sofifa.com/player/243294/tobe-leysen/21...,Tobe Leysen
10375,D. McGeouch,Dylan McGeouch,http://sofifa.com/player/205232/dylan-mcgeouch...,Dylan Mcgeouch
8322,Juan Hernández,Juan Hernández García,http://sofifa.com/player/245219/juan-hernandez...,Juan Hernandez Garcia
7204,C. Golofca,Cătălin Golofca,http://sofifa.com/player/248017/catalin-golofc...,Catalin Golofca
12790,B. Kamau,Bruce Kamau,http://sofifa.com/player/225759/bruce-kamau/21...,Bruce Kamau


## Club

In [16]:
def select_summary(col):
    data=show_missing(df)
    return data[data['variable'] == col]

In [17]:
select_summary('club')

,variable,dtype,count,unique,missing,pc_missing
49,club,object,18979,682,0,0.0


In [18]:
df.loc[:,['club']].sample(10)

,club
7608,\n\n\n\nSK Sturm Graz
16602,\n\n\n\nBohemian FC
16847,\n\n\n\nSagan Tosu
1060,\n\n\n\nMedipol Başakşehir FK
898,\n\n\n\nRacing Club
9287,\n\n\n\nRoyal Excel Mouscron
2369,\n\n\n\nEn Avant de Guingamp
17902,\n\n\n\nBrentford
18272,\n\n\n\nBolton Wanderers
3642,\n\n\n\nUniversidad Católica


<div style=" background-color:#3b3745;text-align:left; padding: 13px 13px; border-radius: 8px; color: white">
<ul>
<li> Spaces at the beginning of club names let's fix them
</ul>
</div>

In [19]:
df['club']=df['club'].apply(lambda x : str(x).lstrip('\n'))

In [20]:
df.loc[:,['club']].sample(10)

,club
16887,Gazişehir Gaziantep F.K.
9372,Al Hilal
17450,Club América
414,Napoli
14671,Oriente Petrolero
13478,Kalmar FF
880,Tigres U.A.N.L.
14667,Rochdale
7886,Fulham
8247,Cusco FC


## Contract

In [21]:
df.loc[:,['contract']].sample(25)

,contract
17964,2018 ~ 2021
5111,2018 ~ 2021
18420,2019 ~ 2020
17376,2020 ~ 2023
2739,2019 ~ 2023
5154,2011 ~ 2021
17227,2018 ~ 2021
10955,2019 ~ 2022
12583,2020 ~ 2020
1187,2020 ~ 2025


In [22]:
def new_colums(dataframe,col):
    contractstart=[]
    contractend=[]
    for d in dataframe['contract']:
        split_data = str(d).split(" ")
        if len(split_data) == 3:
            contractstart.append(split_data[0])
            contractend.append(split_data[2])
        else :
            contractstart.append('N/A')
            contractend.append('N/A')
        
    df_contract = pd.DataFrame(list(zip(contractstart, contractend)),
               columns =['contractstart', 'contractend'])            

    return df_contract

In [23]:
new_colums(df,'contract')

,contractstart,contractend
0,2004,2021
1,2018,2022
2,2014,2023
3,2015,2023
4,2017,2022
...,...,...
18974,2018,2022
18975,2020,2021
18976,2019,2020
18977,2020,2022


In [24]:
df['contractstart'] = new_colums(df,'contract')['contractstart']
df['contractend'] = new_colums(df,'contract')['contractend']

In [25]:
df.loc[:,['contract','contractstart','contractend']].sample(15)

,contract,contractstart,contractend
14673,2020 ~ 2022,2020,2022
7148,2019 ~ 2022,2019,2022
12437,2019 ~ 2021,2019,2021
10424,2014 ~ 2020,2014,2020
6972,2019 ~ 2021,2019,2021
12669,2017 ~ 2022,2017,2022
11684,2016 ~ 2021,2016,2021
5309,"Jun 30, 2021 On Loan",N/A,N/A
13883,2019 ~ 2021,2019,2021
12568,2018 ~ 2021,2018,2021


<div style=" background-color:#3b3745;text-align:left; padding: 13px 13px; border-radius: 8px; color: white">
<ul>
<li>The column has 3 categories of values in the format, '23rd July 2020 on loan’, 'Free' and '2018 ~ 2024’.
<li> Create 2 columns contractstart and contractend 
<li> Free  23rd July 2020 on loan categories replaced with 'Null’
</ul>
</div>

## Joined

In [26]:
select_summary('joined')

,variable,dtype,count,unique,missing,pc_missing
43,joined,object,18979,1869,0,0.0


In [27]:
df.loc[:,['joined']].sample(15)

,joined
7515,"Jul 21, 2018"
5834,"Jul 1, 2019"
11684,"May 23, 2016"
9689,"Jan 1, 2018"
15231,"Jan 1, 2020"
9362,"Jul 1, 2014"
8273,"Aug 28, 2018"
15296,"Jul 16, 2019"
12583,"Feb 1, 2020"
8991,"Jan 5, 2019"


<div style=" background-color:#3b3745;text-align:left; padding: 13px 13px; border-radius: 8px; color: white">
<ul>
<li> data type is wrong we need to change it
</ul>
</div>

In [28]:
df['newjoined'] = df.joined.astype('datetime64')

In [29]:
df.loc[:,['joined','newjoined']].sample(15)

,joined,newjoined
9822,"Jan 7, 2019",2019-01-07
468,"Jul 1, 2011",2011-07-01
6613,"Jan 1, 2020",2020-01-01
3667,"Jul 1, 2019",2019-07-01
3422,"Oct 2, 2020",2020-10-02
12349,"Jul 25, 2019",2019-07-25
3114,"Jul 14, 2017",2017-07-14
16464,"Jan 1, 2018",2018-01-01
15600,"Mar 28, 2019",2019-03-28
3326,"Aug 19, 2020",2020-08-19


## Weakfoot, Skillmoves, and International

In [30]:
select_summary('wf')

,variable,dtype,count,unique,missing,pc_missing
37,wf,object,18979,5,0,0.0


In [31]:
select_summary('sm')

,variable,dtype,count,unique,missing,pc_missing
36,sm,object,18979,5,0,0.0


In [32]:
select_summary('ir')

,variable,dtype,count,unique,missing,pc_missing
33,ir,object,18979,5,0,0.0


In [33]:
df.loc[:,['wf','sm','ir']].head(15)

,wf,sm,ir
0,4 ★,4★,5 ★
1,4 ★,5★,5 ★
2,3 ★,1★,3 ★
3,5 ★,4★,4 ★
4,5 ★,5★,5 ★
5,4 ★,4★,4 ★
6,3 ★,4★,3 ★
7,3 ★,1★,3 ★
8,4 ★,5★,3 ★
9,4 ★,1★,3 ★


<div style=" background-color:#3b3745;text-align:left; padding: 13px 13px; border-radius: 8px; color: white">
<ul>
<li> The columns labeled "wf," "sm," and "ir" each have ratings on a scale of 1 to 5. The data type for these columns was text, and the values all contained the special character "☆" in front of them.
</ul>
</div>

In [34]:
def clean_column_name(dataframe,col):
    clean_col=[]
    for word in dataframe[col]:
        clean_word = re.findall("\d+", word)
        clean_col.append(clean_word)
             
    return  pd.DataFrame(clean_col,columns=['clean_col'])    

In [35]:
df['wf'] = clean_column_name(df,'wf').astype('int')
df['sm'] = clean_column_name(df,'sm').astype('int')
df['ir'] = clean_column_name(df,'ir').astype('int')

In [36]:
df.loc[:,['wf','sm','ir']].head(15)

,wf,sm,ir
0,4,4,5
1,4,5,5
2,3,1,3
3,5,4,4
4,5,5,5
5,4,4,4
6,3,4,3
7,3,1,3
8,4,5,3
9,4,1,3


## Height

In [37]:
select_summary('height')

,variable,dtype,count,unique,missing,pc_missing
49,height,object,18979,62,0,0.0


In [38]:
df.loc[895:902,['height']]

,height
895,"6'3"""
896,"5'7"""
897,"5'10"""
898,"5'9"""
899,183cm
900,183cm
901,180cm
902,182cm


<div style=" background-color:#3b3745;text-align:left; padding: 13px 13px; border-radius: 8px; color: white">
<ul>
<li> The "Height" column entries were formatted as follows: "110cm," and "5'2" (feet & cm).
<li> There are two distinct data items in the height column (feet & inches). These need to be converted into cm.     
<li> foot-->inch--> cm    1 foot 12 inch 1 inch 2.54 cm 
</ul>
</div>

In [39]:
clean_heights=[]
for heights in df.height:
    if 'cm' in heights:
        clean_heights.append(heights.replace("cm",""))
    else:        
        heights=heights.replace('"','').replace("'",".")
        heights=float(heights)* 30.48
        clean_heights.append(int(round(heights)))

In [40]:
clean_heights

['170',
 '187',
 '188',
 '181',
 '175',
 '184',
 '175',
 '191',
 '178',
 '187',
 '193',
 '175',
 '185',
 '199',
 '193',
 '185',
 '184',
 '173',
 '170',
 '168',
 '176',
 '177',
 '188',
 '188',
 '193',
 '187',
 '175',
 '183',
 '176',
 '180',
 '180',
 '173',
 '178',
 '189',
 '179',
 '188',
 '183',
 '181',
 '185',
 '187',
 '189',
 '187',
 '195',
 '180',
 '172',
 '182',
 '188',
 '185',
 '186',
 '192',
 '173',
 '191',
 '165',
 '191',
 '179',
 '194',
 '191',
 '183',
 '173',
 '167',
 '170',
 '182',
 '191',
 '191',
 '176',
 '188',
 '189',
 '188',
 '186',
 '196',
 '175',
 '184',
 '181',
 '186',
 '183',
 '179',
 '175',
 '180',
 '182',
 '181',
 '180',
 '163',
 '186',
 '183',
 '176',
 '190',
 '191',
 '180',
 '174',
 '183',
 '181',
 '191',
 '190',
 '169',
 '183',
 '187',
 '175',
 '178',
 '180',
 '183',
 '185',
 '190',
 '185',
 '181',
 '174',
 '194',
 '181',
 '179',
 '171',
 '171',
 '195',
 '170',
 '184',
 '190',
 '189',
 '184',
 '185',
 '170',
 '172',
 '188',
 '181',
 '178',
 '187',
 '190',
 '178',


In [41]:
df['newheight'] = clean_heights

In [42]:
df.loc[895:902,['height','newheight']]

,height,newheight
895,"6'3""",192
896,"5'7""",174
897,"5'10""",155
898,"5'9""",180
899,183cm,183
900,183cm,183
901,180cm,180
902,182cm,182


##  Weight

In [43]:
select_summary('weight')

,variable,dtype,count,unique,missing,pc_missing
49,weight,object,18979,79,0,0.0


In [44]:
df.loc[895:902,['weight']]

,weight
895,179lbs
896,148lbs
897,170lbs
898,163lbs
899,78kg
900,77kg
901,80kg
902,77kg


<div style=" background-color:#3b3745;text-align:left; padding: 13px 13px; border-radius: 8px; color: white">
<ul>
<li> Entries for the 'Weight' column had the following formats: 65 kg, 112 lb.
<li> These must be changed to kilograms 
<li> 1 lb = 0,45359237 kg
</ul>
</div>

In [45]:
clean_weight=[]
for weights in df.weight:
    if 'kg' in weights:
        clean_weight.append(weights.replace("kg",""))
    else:        
        weights= weights.replace('lbs','')
        weights= int(weights) * 0.453
        clean_weight.append(int(round(weights)))

In [46]:
df['newweightt'] = clean_weight

In [47]:
df.loc[895:902,['weight','newweightt']]

,weight,newweightt
895,179lbs,81
896,148lbs,67
897,170lbs,77
898,163lbs,74
899,78kg,78
900,77kg,77
901,80kg,80
902,77kg,77


## Hits

In [48]:
select_summary('hits')

,variable,dtype,count,unique,missing,pc_missing
1,hits,object,18979,451,2595,13.67


In [49]:
index_list = [7,363,231,1055,692,995,1103,8,34,16203,16204]
df2 = df.loc[df.index[index_list]]
df2.loc[:,['hits']]

,hits
7,120
363,12
231,100
1055,10
692,952
995,979
1103,947
8,1.6K
34,1.5K
16203,NaN


In [50]:
df.hits.fillna("0", inplace = True)

<div style=" background-color:#3b3745;text-align:left; padding: 13px 13px; border-radius: 8px; color: white">
<ul>
<li> The "Hits" field contained numbers but was only stored as text
<li> There are three different formats like: 100, 1.K, and nan value.
<li> I fill na value with 0   
<li> 1K = 1000
</ul>
</div>


In [51]:
clean_hits = []
for hit in df.hits:
    if 'K' in str(hit):
        hit = hit.replace("K", "")
        clean_hits.append(int(float(hit) * 1000)) # 'K' için çarpma işlemi yapılır
    else:
        clean_hits.append(int(hit))


In [52]:
df['newhits'] = (clean_hits)

In [53]:
index_list = [7,363,231,1055,692,995,1103,8,34,16203,16204]
df2 = df.loc[df.index[index_list]]
df2.loc[:,['hits','newhits']]

,hits,newhits
7,120,120
363,12,12
231,100,100
1055,10,10
692,952,952
995,979,979
1103,947,947
8,1.6K,1600
34,1.5K,1500
16203,0,0


## Value, Wage and Release Clause

In [54]:
df.loc[:,['value','wage','releaseclause']].head(10)

,value,wage,releaseclause
0,€103.5M,€560K,€138.4M
1,€63M,€220K,€75.9M
2,€120M,€125K,€159.4M
3,€129M,€370K,€161M
4,€132M,€270K,€166.5M
5,€111M,€240K,€132M
6,€120.5M,€250K,€144.3M
7,€102M,€160K,€120.3M
8,€185.5M,€160K,€203.1M
9,€110M,€260K,€147.7M


<div style=" background-color:#3b3745;text-align:left; padding: 13px 13px; border-radius: 8px; color: white">
<ul>
<li> The columns labeled "Value," "Wage," and "Release clause" had data type errors and values with suffixes in front
<li> The suffixes "M" stood for millions,"K" for thousands, and "€" stood for euros in these columns (currency)
</ul>
</div>

In [55]:
def clean_monetary_data(dataframe,col):
    
    clean_data=[]
    
    for data in dataframe[col]:
        data=data.replace('€','')
        
        if 'K' in data:
            data = data.replace('K','')
            data = float(data) * 1000
            clean_data.append(int(data))
        else:        
            data = data.replace('M','')
            data = float(data) * 1000000
            clean_data.append(int(data))
    return  pd.DataFrame(clean_data,columns=['new_data'])      

In [56]:
df.value = clean_monetary_data(df,'value')
df.wage = clean_monetary_data(df,'wage')
df.releaseclause = clean_monetary_data(df,'releaseclause')

In [57]:
df.loc[:,['value','wage','releaseclause']].head(10)

,value,wage,releaseclause
0,103500000,560000,138400000
1,63000000,220000,75900000
2,120000000,125000,159400000
3,129000000,370000,161000000
4,132000000,270000,166500000
5,111000000,240000,132000000
6,120500000,250000,144300000
7,102000000,160000,120300000
8,185500000,160000,203100000
9,110000000,260000,147700000


# Redundant Columns

In [58]:
df.loc[:,['positions','bestposition']].head(10)

,positions,bestposition
0,"RW, ST, CF",RW
1,"ST, LW",ST
2,GK,GK
3,"CAM, CM",CAM
4,"LW, CAM",LW
5,ST,ST
6,RW,RW
7,GK,GK
8,"ST, LW, RW",ST
9,GK,GK


<div style=" background-color:#3b3745;text-align:left; padding: 13px 13px; border-radius: 8px; color: white">
<ul>
<li> Because each player's "Best Position" is listed in a separate column, the "Positions" column was eliminated. In addition to the additional positions for players who had multiple positions, the information from the "Best Position" field was included in the "Positions" column.
</ul>
</div>

In [59]:
df.loc[:,['id']].head(10)

,id
0,158023
1,20801
2,200389
3,192985
4,190871
5,188545
6,209331
7,212831
8,231747
9,192448


<div style=" background-color:#3b3745;text-align:left; padding: 13px 13px; border-radius: 8px; color: white">
<ul>
<li> Id work like index we dont need ıd.
</ul>
</div>

In [60]:
select_summary('loandateend')

,variable,dtype,count,unique,missing,pc_missing
0,loandateend,object,18979,25,17966,94.66


<div style=" background-color:#3b3745;text-align:left; padding: 13px 13px; border-radius: 8px; color: white">
<ul>
<li> there is a lot of missing data
</ul>
</div>

In [61]:
drop_list = ['id','positions','loandateend','name','longname','photourl','playerurl','contract','height','weight','hits','joined']

In [62]:
final_clean_df=df.drop(drop_list, axis=1)

In [63]:
show_missing(final_clean_df)

,variable,dtype,count,unique,missing,pc_missing
0,nationality,object,18979,164,0,0.0
1,age,int64,18979,29,0,0.0
2,totalstats,int64,18979,1420,0,0.0
3,gkreflexes,int64,18979,70,0,0.0
4,gkpositioning,int64,18979,76,0,0.0
...,...,...,...,...,...,...
67,movement,int64,18979,331,0,0.0
68,ballcontrol,int64,18979,91,0,0.0
69,longpassing,int64,18979,86,0,0.0
70,fkaccuracy,int64,18979,90,0,0.0


In [64]:
final_clean_df

,nationality,age,ova,pot,club,preferredfoot,bov,bestposition,value,wage,...,dri,def,phy,fullname,contractstart,contractend,newjoined,newheight,newweightt,newhits
0,Argentina,33,93,93,FC Barcelona,Left,93,RW,103500000,560000,...,95,38,65,Lionel Messi,2004,2021,2004-07-01,170,72,771
1,Portugal,35,92,92,Juventus,Right,92,ST,63000000,220000,...,89,35,77,C Ronaldo Dos Santos Aveiro,2018,2022,2018-07-10,187,83,562
2,Slovenia,27,91,93,Atlético Madrid,Right,91,GK,120000000,125000,...,90,52,90,Jan Oblak,2014,2023,2014-07-16,188,87,150
3,Belgium,29,91,91,Manchester City,Right,91,CAM,129000000,370000,...,88,64,78,Kevin De Bruyne,2015,2023,2015-08-30,181,70,207
4,Brazil,28,91,91,Paris Saint-Germain,Right,91,LW,132000000,270000,...,94,36,59,Neymar Da Silva Santos Jr,2017,2022,2017-08-03,175,68,595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18974,China PR,21,47,55,Wuhan Zall,Right,49,CB,100000,1000,...,38,48,51,Ao Xia,2018,2022,2018-07-13,178,66,0
18975,England,17,47,67,Oldham Athletic,Right,51,CAM,130000,500000000,...,49,35,45,Ben Hough,2020,2021,2020-08-01,175,65,0
18976,England,18,47,65,Derry City,Right,49,CAM,120000,500000000,...,46,40,53,Ronan Mckinley,2019,2020,2019-03-08,179,74,0
18977,China PR,20,47,57,Dalian YiFang FC,Right,48,ST,100000,2000,...,49,30,44,Zhenao Wang,2020,2022,2020-09-22,175,69,0


In [65]:
#final_clean_df.to_excel('clean_data.xlsx')

<div style=" background-color:#3b3745;text-align:left; padding: 13px 13px; border-radius: 8px; color: white">
<ul>
<li> Our data is ready for analysis. 
<li> Since I don't know the name of the column, there may still be some dirty columns. That's it for now.
<li> I'm open to any kind of offer (volunteer, internship, part-time, full-time)    
</ul>
</div>